In [5]:
import pandas as pd
import numpy as np
import re
from collections import Counter
import plotly.express as px  # (version 4.7.0)
import plotly.graph_objects as go

import dash  # (version 1.12.0) pip install dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

app = dash.Dash(__name__)

In [6]:
df = pd.read_csv('./data/reviews.csv', index_col=0)
df['entities'] = df.expertai_entities.apply(lambda x: [i for i in re.sub('[\]\"  \[\,]','', x).split("'") if i!=''])
df['items'] = df['expertai_sentiment.items'].apply(lambda x: [i for i in re.sub('[\]\"  \[\,]','', x).split("'") if i!=''])
df['phrases'] = df['expertai_mainPhrases'].apply(lambda x: [i for i in re.sub('[\]\"\[\,]','', x).split("'") if i!=''])
df['lemmas'] = df['expertai_mainLemmas'].apply(lambda x: [i for i in re.sub('[\]\"\[\,]','', x).split("'") if i!=''])
df['isAgent'] = df.place_name.apply(lambda x: True if 'Agent' in x else False)


states = sorted(np.append(df.place_state.unique().tolist(),'All'))


In [7]:
app.layout = html.Div([

    html.H1("Web Application Dashboards with Dash", style={'text-align': 'center'}),
    html.Div([
        dcc.Dropdown(id="slct_state",
                     options=[{'label':state, 'value':state} for state in states],
                     multi=False,
                     value='All',
                     style={'width': "40%"}
                     ),
        dcc.RadioItems(
                    id='agent_type',
                    options=[{'label': str(i), 'value': i} for i in [True, False, 'All']],
                    value='All',
#                     labelClassName='isAgent',
    #                 labelStyle={'display': 'inline-block'}
                ) 
    ]),
    
    html.Div([
        dcc.Dropdown(id="slct_type",
                     options=[{'label':state, 'value':state} for state in ['overall', 'service', 'commercial', 'pricing', 'people']],
                     multi=False,
                     value='overall',
                     style={'width': "40%"}
                     ),
    ]),
    html.Div(id='output_container', children=[]),
    html.Br(),

    dcc.Graph(id='my_bee_map', figure={})

])


@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='my_bee_map', component_property='figure')],
    [Input(component_id='slct_state', component_property='value'),
     Input(component_id='agent_type', component_property='value'),
     Input(component_id='slct_type', component_property='value')
    ]
)


def update_graph(slct_state, agent_type, slct_type):
    
    def get_num_items(x, value_to_find:str):
        idx = [index+1 for index, value in enumerate(x) if value == value_to_find]
        return str(dict(Counter(np.array(x)[idx])))

#     print(slct_type)
#     print(type(slct_type))       

    dff = df.copy()
    
    if slct_type != 'overall':
        dff[slct_type] = dff['text'].apply(lambda x: True if slct_type in x else False)
        dff = dff[dff[slct_type] == True]

    
    df_grouped = dff\
    .groupby(['place_id', 'place_state', 'place_formatted_address', 'latitude', 'longitude', 'isAgent'], as_index = False)\
    .agg({
        'items': 'sum', 
        'entities': 'sum', 
        'phrases': 'sum',
        'rating':'mean', 
        'sentiment.overall':'mean',
        'expertai_classification':'count'
    })

    df_grouped['names'] = df_grouped.entities.apply(get_num_items, args={'NPH'})

    if slct_state != 'All':
        df_grouped = df_grouped[df_grouped["place_state"] == slct_state]
    
    if agent_type != 'All':
        df_grouped = df_grouped[df_grouped["isAgent"] == agent_type]
        
   
    mean_rating = round(df_grouped.rating.mean(), 2)
    mean_sentiment = round(df_grouped['sentiment.overall'].mean(), 2)
    
    container = f"The state mean rating: {mean_rating}, mean sentiment: {mean_sentiment}"
    
    fig = px.scatter_mapbox(df_grouped, lat="latitude", lon="longitude", color="sentiment.overall", size='rating',
            color_continuous_scale=px.colors.sequential.Viridis,
            size_max=10, zoom=2,
            text='names',
            mapbox_style = 'carto-positron',
            hover_name = 'place_formatted_address',
            hover_data=["names", "rating"],
    )
    return container, fig


In [8]:
if __name__ == '__main__':
    app.run_server(debug=False, use_reloader=False)


Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
